## Data Science Fianl Project


## The Battle of Neighborhoods 

### Table of Contents
- Introductinon: Business Problem
- Data
- Methodology
- Analysis
- Results
- Conclusion

### Introduction


In this project we will try to predict the top 5 busiest loactions in tokyo to open a restaurant in.

### Data

We have used the wikipedia files and foursquare API to get the locations and the data. Geopy may also be used if in case needed.

### Methodology

We have used the clustering technique and the k-means algorithm to make the predictions.

In [15]:
import sys
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans


In [18]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text
soup = BeautifulSoup(response_obj,'lxml')
Wards_Tokyo_Table = soup.find('table', {'class':'wikitable sortable'})
Name=[]
Kanji = []
Pop = []
Density = []
num = []
flag = []
Area = []
Major_District = []
for row in Wards_Tokyo_Table.findAll("tr"):  
    Ward = row.findAll('td')
    if len(Ward)==8:
        num.append(Ward[0].find(text=True))
        flag.append(Ward[1].findAll('a')) # useless
        Name.append(Ward[2])
        Kanji.append(Ward[3].find(text=True))
        Pop.append(Ward[4])
        Area.append(Ward[5].find(text=True))
        Major_District.append(Ward[7].find(text=True))
Area = ['5100' if x=='0' else x for x in Area]
New_Area = []
for l in range(len(Area)):
    x=Area[l].replace(",","")
    New_Area.append(x)
New_Area=[int(s) for s in New_Area]
new_names = []
for n in range(len(Name)):
    names = Name[n].findAll('a')
    new_names.append(names)
flat_new_names_list = [item for sublist in new_names for item in sublist]
Wards_names= []
for name_wards in flat_new_names_list:
        Wards_names.append(name_wards.get('title'))
replace_names={'Chiyoda, Tokyo':'Chiyoda', 'Chūō, Tokyo':'Chuo', 'Minato, Tokyo':'Minato', 
               'Sumida, Tokyo':'Sumida', 'Koto, Tokyo':'Koto', 'Ōta, Tokyo':'Ota', 'Nakano, Tokyo':'Nakano', 
               'Kita, Tokyo':'Kita', 'Arakawa, Tokyo':'Arakawa', 'Adachi, Tokyo':'Adachi', 'Edogawa, Tokyo':'Edogawa'}
Wards_names1 = [replace_names.get(n1,n1) for n1 in Wards_names]
population = []
for p in range(len(Pop)):
    pops = Pop[p].text[1:9]
    population.append(pops) 
New_population = []
for po in range(len(population)):
    xy=population[po].replace(",","")
    New_population.append(xy)
New_population=[int(s1) for s1 in New_population]
replace_districts = {'Nagatachō':'Nagatacho', 'Hongō':'Hongo', 'Kinshichō':'Kinshicho', 'Ōmori': 'Omori', 
                     'Kōenji':'Koenji', 'Arakawa, Machiya, ':'Arakawa', 'Ayase, ':'Ayase', 'Kasai, Koiwa\n':'Kasai'}
Major_District_names1 = [replace_districts.get(n2,n2) for n2 in Major_District]
df=pd.DataFrame(Wards_names1,columns=['Ward'])
df['Area_SqKm'] = New_Area
df['Population'] = New_population
df['Major_District'] = Major_District_names1
df.index = np.arange(1, len(df) + 1) # reset the index so that it starts from 1. 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Tokyo_explorer")
df['Major_Dist_Coord']= df['Major_District'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Major_Dist_Coord'].apply(pd.Series)
df.drop(['Major_Dist_Coord'], axis=1, inplace=True)
Lat_list = df['Latitude'].tolist()
Long_list = df['Longitude'].tolist()
replace_latitudes = {32.5093796:35.7088, 23.0131338:35.6722, -38.9047057:35.5884, -5.3498001:35.6634}
replace_longitudes = {-116.2970014:139.7601, -80.8328748:139.8061, 175.7552111:139.7279, 21.424098:139.8731}
latitudes_new = [replace_latitudes.get(n3,n3) for n3 in Lat_list]
longtitudes_new = [replace_longitudes.get(n4,n4) for n4 in Long_list]
Tokyo_df = df.drop(['Latitude', 'Longitude'], axis=1)
Tokyo_df['Dist_Latitude'] = latitudes_new
Tokyo_df['Dist_Longitude'] = longtitudes_new
Tokyo_df
Tokyo_df.to_csv('Tokyo_df_Coord.csv')


In [19]:
Tokyo_df = pd.read_csv('Tokyo_df_Coord.csv', index_col=[0]) 
Tokyo_df.head()

,Ward,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469
2,Chuo,14460,147620,Nihonbashi,35.684058,139.774501
3,Minato,12180,248071,Odaiba,35.619050,139.779364
4,Shinjuku,18620,339211,Shinjuku,35.693763,139.703632
5,Bunkyō,19790,223389,Hongo,35.175376,137.013476


In [35]:
price_df = pd.DataFrame({'Ward':["Chiyoda-Ku","Chuo-Ku","Minato-Ku","Shinjuku-Ku","Bunkyo-Ku","Taito-Ku","Sumida-Ku","Koto-Ku","Shinagawa-Ku","Meguro-Ku","Ota-Ku","Setagaya-Ku","Shibuya-Ku","Nakano-Ku","Suginami-Ku","Toshima-Ku","Kita-Ku","Arakawa-Ku","Itabashi-Ku","Nerima-Ku","Adachi-Ku","Katsushika-Ku","Edogawa-Ku","Hachioji-Shi","Tachikawa-Shi","Musashino-Shi","Mitaka-Shi","Ome-Shi","Fuchu-Shi","Akishima-Shi","Chofu-Shi","Machida-Shi","Koganei-Shi",
"Kodaira-Shi","Hino-Shi","Higashimurayama-Shi","Kokubunji-Shi","Kunitachi-Shi","Fussa-Shi","Komae-Shi","Higashiyamato-Shi","Kiyose-Shi","Higashikurume-Shi","Musashimurayama-Shi","Tama-Shi","Inagi-Shi","Hamura-Shi","Akiruno-Shi",
"Nishitokyo-Shi","Nishitama-Gun",
"Oshima-Machi","Niijima-Mura","Kozushima-Mura","Ogasawara-Mura"],'Average_Price':[1703896,2798951,2142315,928783,923184,932778,561692,553600,707708,834455,524360,628816,1284956,580027,551316,656064,465679,415566,424673,385951,282614,310759,313333,131744,231999,541458,356517,88085,277748,184025,360582,158877,313586,220458,202936,174841,252024,302179,156312,293152,155941,166731,200217,116506,179714,216463,161173,79763,266500,67686,12108,11119,8166,29977,
]})
price_df.head()

,Ward,Average_Price
0,Chiyoda-Ku,1703896
1,Chuo-Ku,2798951
2,Minato-Ku,2142315
3,Shinjuku-Ku,928783
4,Bunkyo-Ku,923184


In [36]:
pd.options.mode.chained_assignment = None
ward_list = ['Chiyoda-Ku', 'Chuo-Ku', 'Shinjuku-Ku', 'Shibuya-Ku', 'Shinagawa-Ku']
price_df_selected = price_df.loc[price_df['Ward'].isin(ward_list)]
price_df_selected.index = np.arange(1, len(price_df_selected) + 1)
price_df_selected['Ward']=price_df_selected['Ward'].map(lambda y: str(y)[:-3]) 
price_df_selected

,Ward,Average_Price
1,Chiyoda,1703896
2,Chuo,2798951
3,Shinjuku,928783
4,Shinagawa,707708
5,Shibuya,1284956


In [37]:
ward_list1 = ['Chiyoda', 'Chuo', 'Shinjuku', 'Shibuya', 'Shinagawa']
Tokyo_df_selected = Tokyo_df.loc[Tokyo_df['Ward'].isin(ward_list1)]
Tokyo_df_selected

,Ward,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469
2,Chuo,14460,147620,Nihonbashi,35.684058,139.774501
4,Shinjuku,18620,339211,Shinjuku,35.693763,139.703632
9,Shinagawa,17180,392492,Shinagawa,35.599252,139.738910
13,Shibuya,15080,227850,Shibuya,35.664596,139.698711


In [38]:
Tokyo_df_selected_final = Tokyo_df_selected.merge(price_df_selected, on='Ward', how='left')
Tokyo_df_selected_final.index = np.arange(1, len(Tokyo_df_selected_final)+1)
Tokyo_df_selected_final.to_csv('Tokyo_5_Wards.csv')
Tokyo_df_selected_final

,Ward,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude,Average_Price
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469,1703896
2,Chuo,14460,147620,Nihonbashi,35.684058,139.774501,2798951
3,Shinjuku,18620,339211,Shinjuku,35.693763,139.703632,928783
4,Shinagawa,17180,392492,Shinagawa,35.599252,139.738910,707708
5,Shibuya,15080,227850,Shibuya,35.664596,139.698711,1284956


In [ ]:
from geopy.geocoders import Nominatim
address = 'Tokyo'
geolocator = Nominatim(user_agent="Tokyo_explorer")
location = geolocator.geocode(address)
Tokyo_latitude = location.latitude
Tokyo_longitude = location.longitude
print('The geograpical coordinates of Tokyo are {}, {}.'.format(Tokyo_latitude, Tokyo_longitude))
import folium
Tokyo_5Wards = folium.Map(location=[Tokyo_latitude, Tokyo_longitude], zoom_start=12)
for lat, lng, label in zip(Tokyo_df_selected_final['Dist_Latitude'], Tokyo_df_selected_final['Dist_Longitude'], 
                           Tokyo_df_selected_final['Major_District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='magenta',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(Tokyo_5Wards)  
Tokyo_5Wards

In [43]:
CLIENT_ID = 'YU3XPNNLQ2Q24N2IZZNQLC301R1110KUOH32RBFFCI5ZHIAY'
CLIENT_SECRET = 'L0051GEIRVYKRDPLYBDAIDDD3UYLQS3QYHLUZSZUNPZFT5OO' 
VERSION = '20190325'

print('My credentails:')
print('My CLIENT_ID: ' + CLIENT_ID)
print('My CLIENT_SECRET:' + CLIENT_SECRET)

radius = 1000
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District','Dist_Latitude','Dist_Longitude','Venue','Venue_Lat','Venue_Long','Venue_Category']
    return(nearby_venues)
Tokyo_5_Dist_Venues = getNearbyVenues(names=Tokyo_df_selected_final['Major_District'],
                                   latitudes=Tokyo_df_selected_final['Dist_Latitude'],
                                   longitudes=Tokyo_df_selected_final['Dist_Longitude']
                                  )

My credentails:
My CLIENT_ID: YU3XPNNLQ2Q24N2IZZNQLC301R1110KUOH32RBFFCI5ZHIAY
My CLIENT_SECRET:L0051GEIRVYKRDPLYBDAIDDD3UYLQS3QYHLUZSZUNPZFT5OO
Nagatacho
Nihonbashi
Shinjuku
Shinagawa
Shibuya


In [44]:
Tokyo_5_Dist_Venues_Top10 = Tokyo_5_Dist_Venues['Venue_Category'].value_counts()[0:10].to_frame(name='frequency')
Tokyo_5_Dist_Venues_Top10=Tokyo_5_Dist_Venues_Top10.reset_index()
Tokyo_5_Dist_Venues_Top10.rename(index=str, columns={"index": "Venue_Category", "frequency": "Frequency"}, inplace=True)
Tokyo_5_Dist_Venues_Top10

,Venue_Category,Frequency
0,Japanese Restaurant,32
1,Ramen Restaurant,26
2,BBQ Joint,25
3,Sake Bar,19
4,Café,17
5,Bar,15
6,Chinese Restaurant,14
7,Convenience Store,14
8,Coffee Shop,13
9,Hotel,13


In [47]:
Tokyo_5_Dist_Venues= Tokyo_5_Dist_Venues.groupby('District').mean().reset_index()
Tokyo_5_Dist_Venues.index = np.arange(1, len(Tokyo_5_Dist_Venues)+1)
Tokyo_5_Dist_Venues

,District,Dist_Latitude,Dist_Longitude,Venue_Lat,Venue_Long
1,Nagatacho,35.675618,139.743469,35.674331,139.738711
2,Nihonbashi,35.684058,139.774501,35.684289,139.773012
3,Shibuya,35.664596,139.698711,35.662142,139.698333
4,Shinagawa,35.599252,139.738910,35.602061,139.737633
5,Shinjuku,35.693763,139.703632,35.693073,139.703151


In [48]:
kclusters = 3
Tokyo_grouped_clustering = Tokyo_5_Dist_Venues.drop('District', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Tokyo_grouped_clustering)
print ("Check the 5 Cluster labels :",  kmeans.labels_[0:5])

Check the 5 Cluster labels : [2 2 0 1 0]


In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 4
indicators = ['st', 'nd', 'rd']
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

District_top10_venues_sorted = pd.DataFrame(columns=columns)
District_top10_venues_sorted['District'] = Tokyo_5_Dist_Venues['District']

for ind in np.arange(Tokyo_5_Dist_Venues.shape[0]):
    District_top10_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Tokyo_5_Dist_Venues.iloc[ind, :], num_top_venues)

District_top10_venues_sorted.index = np.arange(1, len(District_top10_venues_sorted)+1)
District_top10_venues_sorted.head(5)


,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
1,Nagatacho,Dist_Longitude,Venue_Long,Dist_Latitude,Venue_Lat
2,Nihonbashi,Dist_Longitude,Venue_Long,Venue_Lat,Dist_Latitude
3,Shibuya,Dist_Longitude,Venue_Long,Dist_Latitude,Venue_Lat
4,Shinagawa,Dist_Longitude,Venue_Long,Venue_Lat,Dist_Latitude
5,Shinjuku,Dist_Longitude,Venue_Long,Dist_Latitude,Venue_Lat


In [55]:
District_top10_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)
Tokyo_5_Dist_Coordinate_Cluster_merged = Tokyo_df_selected_final
Tokyo_5_Dist_Coordinate_Cluster_merged = Tokyo_5_Dist_Coordinate_Cluster_merged.join \
                                        (District_top10_venues_sorted.set_index('District'), on='Major_District')

Tokyo_5_Dist_Coordinate_Cluster_merged.head()

,Ward,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude,Average_Price,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469,1703896,2,Dist_Longitude,Venue_Long,Dist_Latitude,Venue_Lat
2,Chuo,14460,147620,Nihonbashi,35.684058,139.774501,2798951,2,Dist_Longitude,Venue_Long,Venue_Lat,Dist_Latitude
3,Shinjuku,18620,339211,Shinjuku,35.693763,139.703632,928783,0,Dist_Longitude,Venue_Long,Dist_Latitude,Venue_Lat
4,Shinagawa,17180,392492,Shinagawa,35.599252,139.738910,707708,1,Dist_Longitude,Venue_Long,Venue_Lat,Dist_Latitude
5,Shibuya,15080,227850,Shibuya,35.664596,139.698711,1284956,0,Dist_Longitude,Venue_Long,Dist_Latitude,Venue_Lat


In [57]:
map_restaurants10 = folium.Map(location=[Tokyo_latitude, Tokyo_longitude], tiles='cartodbpositron', 
                               attr="<a href=https://github.com/python-visualization/folium/>Folium</a>")
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
Districts = ['Nagatacho', 'Nihonbashi', 'Shinjuku', 'Shinagawa', 'Shibuya']
list_rest_no = [56, 45, 28, 35, 29]
for lat, lon, poi, cluster in zip(Tokyo_5_Dist_Coordinate_Cluster_merged['Dist_Latitude'], 
                                  Tokyo_5_Dist_Coordinate_Cluster_merged['Dist_Longitude'], 
                                  Tokyo_5_Dist_Coordinate_Cluster_merged['Major_District'], 
                                  Tokyo_5_Dist_Coordinate_Cluster_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=list_rest_no[Districts.index(poi)]*0.5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_restaurants10)
       
map_restaurants10
